In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import joblib
import mlflow
import mlflow.keras
import mlflow.sklearn

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [3]:
x_train_cnn = np.expand_dims(x_train, -1)  # shape: (N, 28, 28, 1)
x_test_cnn = np.expand_dims(x_test, -1)

In [4]:
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [7]:
mlflow.set_experiment("Fashion_MNIST_Classifier")

with mlflow.start_run(run_name="CNN_Fashion_MNIST"):
    cnn_model.fit(x_train_cnn, y_train, epochs=5, batch_size=64, validation_split=0.1)
    test_loss, test_acc = cnn_model.evaluate(x_test_cnn, y_test)
    
    mlflow.log_param("model_type", "CNN")
    mlflow.log_metric("test_accuracy", test_acc)
    mlflow.keras.log_model(cnn_model, "cnn_model")
    
    # Save manually too
    cnn_model.save("model_cnn.h5")


2025/06/01 21:14:50 INFO mlflow.tracking.fluent: Experiment with name 'Fashion_MNIST_Classifier' does not exist. Creating a new experiment.


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.7330 - loss: 0.7441 - val_accuracy: 0.8652 - val_loss: 0.3815
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8738 - loss: 0.3546 - val_accuracy: 0.8757 - val_loss: 0.3339
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8894 - loss: 0.3030 - val_accuracy: 0.8887 - val_loss: 0.3014
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.9017 - loss: 0.2715 - val_accuracy: 0.8957 - val_loss: 0.2888
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9121 - loss: 0.2440 - val_accuracy: 0.9013 - val_loss: 0.2726
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8955 - loss: 0.2952


2025/06/01 21:15:48 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/06/01 21:15:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [8]:
print(f"CNN Test Accuracy: {test_acc:.4f}")

CNN Test Accuracy: 0.8950


In [9]:
x_train_flat = x_train.reshape(-1, 28 * 28)
x_test_flat = x_test.reshape(-1, 28 * 28)

In [10]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_flat)
x_test_scaled = scaler.transform(x_test_flat)

In [11]:
x_train_small, _, y_train_small, _ = train_test_split(
    x_train_scaled, y_train, train_size=10000, stratify=y_train, random_state=42)

svm = SVC(kernel='rbf', gamma='scale')

In [12]:
with mlflow.start_run(run_name="SVM_Fashion_MNIST"):
    svm.fit(x_train_small, y_train_small)
    y_pred = svm.predict(x_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    
    mlflow.log_param("model_type", "SVM")
    mlflow.log_param("kernel", "rbf")
    mlflow.log_metric("test_accuracy", acc)
    mlflow.sklearn.log_model(svm, "svm_model")

    joblib.dump(svm, "model_svm.pkl")
    joblib.dump(scaler, "scaler_svm.pkl")  # Save scaler for preprocessing during inference

print("SVM Accuracy:", acc)

2025/06/01 21:18:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


SVM Accuracy: 0.8522
